# Crop Health Analysis 2019

In 2019, the US Midwest was impacted by significant and prolonged rainfall, delaying the planting of corn, soybeans, and other summer crops.  In this notebook we will use Descartes Labs `Workflows` to investigate the impact on overall crop production.

You can run the following cells using `Shift-Enter`.

## Import packages

In [ ]:
# keep logging quiet
import logging
logging.getLogger().setLevel(logging.INFO)
logging.captureWarnings(True)

In [ ]:
# import packages
from descarteslabs import workflows as wf
import IPython
import ipyleaflet
import ipywidgets
import arrow
import matplotlib.pyplot as plt
%matplotlib inline

## Visualizing crop health
When crops such as corn and soybeans are healthy and growing, they tend to heavily absorb red light used for photosynthesis. At the same time, they tend to reflect near-infrared light which would otherwise heat the plant to too high of a temperature. Because of this, agronomists tend to measure what they call the normalized difference vegetation index, or NDVI. This is defined by the ratio of (near-infrared - red) / (near-infrared + red), which is a nice quantity because it is bounded by `-1 <= NDVI <= 1`.

As part of the Descartes Labs Data Refinery, incoming daily measurements by the MODIS instrument on the Aqua and Terra satellites from NASA capture the relative strength of the red and near-infrared wavelengths. However, because clouds tend to obscure an average of 30-40% of the Earth's landmass every day, it can be difficult to get a clear shot. Below is shown 2 days of MODIS visual (red-green-blue) and NDVI.

In [ ]:
# Define Red-Green-Blue imagery from MODIS.
# In these few lines of code below, we've abstracted searching a Petascale dataset, rasterizing the matching imagery from
# that dataset, picking bands, scales, etc, and creating a visualization that you can interactively explore.

(
    wf.ImageCollection.from_id(
        "modis:09:v2", start_datetime="2019-06-01", end_datetime="2019-06-03"
    )
    .pick_bands("red green blue")
    .mean(axis="images")
    .visualize("MODIS RGB", scales=[[0, 0.4], [0, 0.4], [0, 0.4]])
)
wf.map.center = 40.5, -88.5
wf.map.zoom = 7
# layer_controller = wf.interactive.LayerController(wf.map.map, position='bottomleft')
wf.map.layout.height = "800px"
wf.map

## Data Refinement
To counteract the clouds, we use the MOD13Q1 data product from NOAA: the maximum MODIS NDVI over a 16-day window.  In the next set of maps we show the raw NDVI values, in which we can clearly see the contamination from clouds / shadows. Our refined maximum-NDVI data product displays a cloud-free quantity that can now be input to machine learning models.

As we run the commands below, these new imagery layers will appear in the map above.  You can always create a new instance of this interactive map by executing the command `wf.map` in this notebook.

In [ ]:
# Define Raw NDVI from 2019
(
    wf.ImageCollection.from_id(
        "modis:09:v2", start_datetime="2019-06-01", end_datetime="2019-06-03"
    )
    .pick_bands("ndvi")
    .mean(axis="images")
).visualize("2019 Raw NDVI", scales=[[0, 1]], colormap="RdYlGn")

In [ ]:
# Define Refined NDVI from 2019
ndvi_2019 = (
    wf.ImageCollection.from_id(
        "modis:mod13q1:006", start_datetime="2019-06-09", end_datetime="2019-06-11"
    )
    .pick_bands("ndvi")
    .mean(axis="images")
)
ndvi_2019.visualize("2019 Refined NDVI", scales=[[0, 1]], colormap="RdYlGn")

### If we now compare to 2018...
The only parameter we have to change here is the start day of the 16-day window, now 2018-06-09.

In [ ]:
# Define Refined NDVI from 2018
ndvi_2018 = (
    wf.ImageCollection.from_id(
        "modis:mod13q1:006", start_datetime="2018-06-09", end_datetime="2018-06-11"
    )
    .pick_bands("ndvi")
    .mean(axis="images")
)
ndvi_2018.visualize("2018 Refined NDVI", scales=[[0, 1]], colormap="RdYlGn")

### Derived layers

While these first sets of visualizations are simply displaying data that exists in the platform, we can also create derived layers on the fly. Here we can create the difference between 2019 and 2018. Blue areas are healthier in 2019, while brown areas are less healthy in 2019.  The effects of a prolonged wet spring in 2019 are clear throughout the midwest.

In [ ]:
# Define the difference between Refined NDVI in 2019 and 2018
(ndvi_2019 - ndvi_2018).visualize(
    "NDVI Delta, 2019-2018", scales=[[-0.5, 0.5]], colormap="BrBG"
)

### Explore crop health interactively

Below we will define a custom ipywidget, `Auger`, that will allow us to draw an AOI in the map and then view the NDVI time series for that AOI in 2018 and 2019.  You can draw the AOI using the box-draw tool on the left side of the map.  This triggers a computation of historical NDVI using the Descartes Labs `Workflows` backend.

In [ ]:
from utils import Auger

ndvi_timeseries = Auger(
    wf.map,
    variable=wf.ImageCollection.from_id(
        "modis:mod13q1:006", start_datetime="2018-01-01", end_datetime="2020-01-01"
    ).pick_bands("ndvi"),
)

fig_output = ipywidgets.Output()
fig_widget = ipyleaflet.WidgetControl(widget=fig_output, position="bottomright")
wf.map.add_control(fig_widget)


@wf.map.output_log.capture(clear_output=True)
def do_plot(*args, **kwargs):
    fig, ax = plt.subplots(figsize=[6, 6])
    ax.cla()
    ax.set_visible(True)
    df = ndvi_timeseries.df
    df["ndvi"] = df["ndvi"].astype("float")
    df["doy"] = df.apply(lambda row: row.dates.dayofyear, axis=1)
    df["year"] = df.apply(lambda row: row.dates.year, axis=1)
    for yr in df["year"].unique():
        _ = df[df["year"] == yr].plot("doy", "ndvi", ax=ax, label="{}".format(yr))
    _ = ax.set_xlabel("Day-of-year")
    _ = ax.set_ylabel("NDVI")
    _ = ax.set_xlim(120, 300)

    with fig_output:
        IPython.display.display(fig)

    return ""


ndvi_timeseries.draw_control.on_draw(do_plot)

### If desired, return the NDVI time series as a pandas `dataframe`.
You must first draw an AOI in the map above.  Then uncomment and run the line below.

In [ ]:
# ndvi_timeseries.df